In [1]:
from discrete_motif import DiscreteGrnMotif
import discrete_motif_functions as functions
import discrete_motif_operations as operations
import discrete_motif_measures as measures
import discrete_motif_generator as generator
import discrete_motif_plotting as visualize
from IPython.display import HTML, display
import tabulate

# Example 1: X-OR

In this example, we demonstrate an X-OR motif, where gene $A_{t=0}$ and $B_{t=0}$ set $B_{t=1}$.

First, we we create an empty motif with 1 variable and 2 possible states.

In [2]:
motif_xor = DiscreteGrnMotif(1, 2, 'random')

We then enter our motif by adding a second gene, defining a rule, setting the decision rule in case of conflicting rules, and then contructing the GRN-object.

In [3]:
motif_xor.grn_vars["gene_cnt"] = 2
motif_xor.grn_vars["conflict_rule"] = 'totaleffect'
motif_xor.append_rule([0, 1], [1], functions.xor)
motif_xor.construct_grn()

We can now evaluate our motif, one timestep at a time. We do this once so we can look at results.

In [4]:
motif_xor.evaluate_motif(genes=[0, 1])

We can look at marginalized probability distributions of the system by accessing the states list, which is appended to every time you run the evaluate_motif function. We can also look at the transition state table. Note that the X-OR determines the increase of gene 2, meaning that in the (1, 1) case we still get a 1 returned (namely 1 + 0).

In [5]:
print("The states: ")
for state in motif_xor.states:
    print(state)
table = visualize.state_transition_table(motif_xor, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

The states: 
[[ 0.14249939  0.19654576]
 [ 0.49132692  0.16962793]]
[[ 0.14249939  0.19654576]
 [ 0.0  0.66095486]]


Gene 0 t=0,Gene 1 t=0,Gene 0 t=1,Gene 1 t=1
0,0,0,0
0,1,0,1
1,0,1,1
1,1,1,1


We can perform several analysis methods on this motif. We find that there is a significant amount of synergy in this system, as we expect. It is also about the amount we should find: there is 2 bits of information mutual information, and 1 bit synergy.

In [6]:
print("The mutual information: ")
print(str(measures.mutual_information(motif_xor, [0]))+" component 1")
print(str(measures.mutual_information(motif_xor, [1]))+" component 2")
print(str(measures.mutual_information(motif_xor))+" total")
print("The WMS information: ")
print(measures.synergy_wms(motif_xor))
print("The SRV synergy: ")
print(measures.synergy_quax(motif_xor))
print("The UII synergy: ")
print(measures.synergy_uii(motif_xor))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_xor))

The mutual information: 
0.923902040106 component 1
0.404625297539 component 2
1.25670589282 total
The WMS information: 
0.13049120394
The SRV synergy: 


jointpdf/jointpdf.py:2645: UserWarning: all 30 optimizations using minimize() failed...?!. Will now skip this sample in synergistic_information.
  warnings.warn(str(e) + '. Will now skip this sample in synergistic_information.')


0
The UII synergy: 
1.32852733765
The middle ground synergy approximation: 
0.527196622023


We can examine the decay of mutual information over multiple timesteps with a build-in function. In this case this is not very interesting.

In [ ]:
print(measures.mi_decay(motif_xor))

Finally, we can reset the state, nudge 2 genes with a 0.5-impact nudge. After doing so, we can assess the nudge impact by comparing the state after our initial timestep, with the state after nudging and moving ahead one timestep.

In [ ]:
motif_xor.reset_to_state(0)
operations.nudge_variable(motif_xor, 2, 0.5)
motif_xor.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_xor.states[1], motif_xor.states[-1]))

# Example 2: AND

We can perform the same steps with an AND-gate.

In [7]:
motif_and = DiscreteGrnMotif(1, 2, 'random')
motif_and.grn_vars["gene_cnt"] = 2
motif_and.grn_vars["conflict_rule"] = 'totaleffect'
motif_and.append_rule([0, 1], [1], functions.plus_and)
motif_and.construct_grn()
motif_and.evaluate_motif(genes=[0, 1])
print("The states: ")
for state in motif_and.states:
    print(state)
table = visualize.state_transition_table(motif_and, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

The states: 
[[ 0.10691352  0.36957434]
 [ 0.29993406  0.22357808]]
[[ 0.47648786  0.0]
 [ 0.0  0.52351214]]


Gene 0 t=0,Gene 1 t=0,Gene 0 t=1,Gene 1 t=1
0,0,0,0
0,1,0,1
1,0,1,0
1,1,1,1


After initiating, we can do our synergy and nudge impact analysis.

In [8]:
print("The mutual information: ")
print(measures.mutual_information(motif_and))
print("The WMS information: ")
print(measures.synergy_wms(motif_and))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_and))
print("The memory: ")
print(measures.mi_decay(motif_and))
motif_and.reset_to_state(0)
operations.nudge_variable(motif_and, 2, 0.5)
motif_and.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_and.states[1], motif_and.states[-1]))

The mutual information: 
0.998404308201
The WMS information: 
-0.0466919405453
The middle ground synergy approximation: 
0.475856183828
The memory: 
[1.87983593271784096, 1.8798359327178409601, 0.99840430820125569667, 0.99840430820125569678, 0.99840430820125569678, 0.99840430820125569678, 0.99840430820125569678, 0.99840430820125569678, 0.99840430820125569678]
The nudge impact: 
0.0373908750324


# Example 3: random motifs

To do randomized experiments, we also support the generation of random motifs. These motifs incorporate random relationships between genes, drawn from a library of all possible relations in a GRN motif.

First, we generate a single random motif of 2 genes, with 2 relationships.

In [9]:
motifs, _ = generator.generate_motifs(samplesize=2, no_nodes=2, indegree=1, conflict_rule='totaleffect')
motif_rand = motifs[0]
motif_rand.evaluate_motif(genes=[0, 1])
print("The states: ")
for state in motif_rand.states:
    print(state)
table = visualize.state_transition_table(motif_rand, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

The states: 
[[ 0.23450565  0.12478105]
 [ 0.62156773  0.01914557]]
[[ 0.3592867  0.0]
 [ 0.0  0.6407133]]


Gene 0 t=0,Gene 1 t=0,Gene 0 t=1,Gene 1 t=1
0,0,0,0
0,1,0,1
1,0,1,0
1,1,1,1


We then can do our usual analysis.

In [10]:
print("The rules: ")
print(motif_rand.grn_vars["rules"])
print("The correlations: ")
print(motif_rand.grn_vars["correlations"])
print("The mutual information: ")
print(measures.mutual_information(motif_rand))
print("The WMS information: ")
print(measures.synergy_wms(motif_rand))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_rand))
print("The memory: ")
print(measures.mi_decay(motif_rand))
motif_rand.reset_to_state(0)
operations.nudge_variable(motif_rand, 2, 0.5)
motif_rand.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_rand.states[1], motif_rand.states[-1]))

The rules: 
[{'inputs': [0, 1], 'rulefunction': <function incl_or at 0x7f790671ad70>, 'outputs': [1]}, {'inputs': [0], 'rulefunction': <function minus at 0x7f790671aaa0>, 'outputs': [1]}]
The correlations: 
[[ 0.52934716 -0.25424497]
 [-0.25424497  0.47065284]]
The mutual information: 
0.942089506025
The WMS information: 
-0.0677686964044
The middle ground synergy approximation: 
0.43716040481
The memory: 
[1.4009827269287818117, 1.4009827269287818117, 0.9420895060251391227, 0.9420895060251391227, 0.9420895060251391227, 0.9420895060251391227, 0.9420895060251391227, 0.9420895060251391227, 0.9420895060251391227]
The nudge impact: 
0.27237925591


# Example 4: multivalued logic (3 levels)

The SRV measure should work better for higher-valued logic, we try this out too. We should check the correlation matrix generated, this should be something sensible.

In [13]:
motifs, _ = generator.generate_motifs(samplesize=1, no_nodes=3, indegree=3, conflict_rule='totaleffect', numvalues=3)
motif_rand = motifs[0]
motif_rand.evaluate_motif()
print("The states: ")
for state in motif_rand.states:
    print(state)
print("The correlations: ")
print(motif_rand.grn_vars["correlations"])

table = visualize.state_transition_table(motif_rand, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

The states: 
[[[ 0.01536798  0.06669316  0.03739554]
  [ 0.01839814  0.00805939  0.0515968 ]
  [ 0.00773401  0.00563619  0.05947582]]

 [[ 0.02734127  0.01465545  0.00341052]
  [ 0.018251    0.06334616  0.03066262]
  [ 0.06018396  0.07508282  0.08012377]]

 [[ 0.03967852  0.01473776  0.07918418]
  [ 0.03726042  0.0417156   0.06946362]
  [ 0.01271332  0.04059785  0.02123415]]]
[[[ 0.015367978  0.0  0.0]
  [ 0.0  0.0  0.0]
  [ 0.0  0.037395537  0.066693158]]

 [[ 0.027341265  0.0  0.086060992]
  [ 0.0  0.0  0.085669935]
  [ 0.0  0.0  0.0]]

 [[ 0.0  0.19421936  0.48725177]
  [ 0.0  0.0  0.0]
  [ 0.0  0.0  0.0]]]
The correlations: 
[[ 0.          0.90285899 -0.74721148]
 [ 0.90285899  0.         -0.82760596]
 [-0.74721148 -0.82760596  0.        ]]


Gene 0 t=0,Gene 1 t=0,Gene 2 t=0,Gene 0 t=1,Gene 1 t=1,Gene 2 t=1
0,0,0,0,0,0
0,0,1,0,0,1
0,0,2,0,0,2
0,1,0,2,2,0
0,1,1,1,0,0
0,1,2,1,0,2
0,2,0,2,2,1
0,2,1,2,1,1
0,2,2,2,0,1
1,0,0,1,0,0


We then can do our usual analysis.

In [14]:
print("The rules: ")
print(motif_rand.grn_vars["rules"])
print("The mutual information: ")
print(measures.mutual_information(motif_rand))
print("The WMS information: ")
print(measures.synergy_wms(motif_rand))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_rand))
print("The memory: ")
print(measures.mi_decay(motif_rand))
motif_rand.reset_to_state(0)
operations.nudge_variable(motif_rand, 2, 0.5)
motif_rand.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_rand.states[1], motif_rand.states[-1]))

The rules: 
[{'inputs': [0, 1], 'rulefunction': <function min_and at 0x7f790671ac08>, 'outputs': [0]}, {'inputs': [1, 2], 'rulefunction': <function neg_incl_or at 0x7f790671ade8>, 'outputs': [0]}, {'inputs': [1], 'rulefunction': <function superplus at 0x7f790671a9b0>, 'outputs': [0]}, {'inputs': [1, 2], 'rulefunction': <function xor at 0x7f790671ac80>, 'outputs': [2]}, {'inputs': [2], 'rulefunction': <function superminus at 0x7f790671ab18>, 'outputs': [1]}, {'inputs': [0, 1], 'rulefunction': <function xor at 0x7f790671ac80>, 'outputs': [1]}, {'inputs': [1], 'rulefunction': <function plus at 0x7f790671aa28>, 'outputs': [2]}, {'inputs': [0, 2], 'rulefunction': <function neg_xor at 0x7f790671acf8>, 'outputs': [1]}, {'inputs': [0, 1], 'rulefunction': <function neg_excl_or at 0x7f790671aed8>, 'outputs': [2]}]
The mutual information: 
2.24518989263
The WMS information: 
0.653925257239
The middle ground synergy approximation: 
0.867887948122
The memory: 
[4.4240109875115210098, 4.424010987511

The SRV synergy should be easier to compute in this case.

In [ ]:
print("The SRV synergy: ")
print(measures.synergy_quax(motif_rand))